In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

AmExpert 2021 – Machine Learning Hackathon

LB: 115/2931
SCORE : 0.5295 Using Catboost ,0.4926 Using KNN


In [2]:
sample_submission=pd.read_csv("../input/amexpert-2021-challenge/sample_submission_kF044ur.csv")
sample_submission.head()

In [3]:
train=pd.read_csv("../input/amexpert-2021-challenge/train_go05W65.csv")
train.head()

In [4]:
train['Product_Holding_B1']=train['Product_Holding_B1'].apply(eval)
train.head()

In [5]:
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [6]:
to_1D(train['Product_Holding_B1']).value_counts()

In [7]:
train1=train['Product_Holding_B1'].apply(pd.Series)
train1.head()

In [8]:
train1.isnull().sum()

In [9]:
train1.drop([4,5,6,7],axis=1,inplace=True)
train1.head()

In [10]:
train1=train1.fillna('0')
train1.head()

In [11]:
result = pd.concat([train, train1], axis=1)
result

In [ ]:
#train['len_Product_Holding_B1']=train['Product_Holding_B1'].apply(lambda x: len(x.split(',')))
#train['len_Product_Holding_B2']=train['Product_Holding_B2'].apply(lambda x: len(x.split(',')))
#train.head()

In [12]:
result.info()

In [14]:
result.isnull().sum()

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
sns.countplot(result.Gender)

In [19]:
sns.countplot(result.Is_Active)

In [20]:
sns.countplot(result.City_Category)

In [21]:
sns.countplot(result.Customer_Category)

In [ ]:

#pd.options.display.max_rows = 495
#train.Product_Holding_B2.value_counts()

In [22]:
result.Product_Holding_B1.value_counts()

In [24]:
sns.histplot(result.Age)

In [25]:
sns.distplot(result.Vintage)

In [26]:
test=pd.read_csv("../input/amexpert-2021-challenge/test_VkM91FT.csv")
test.head()

In [27]:
test['Product_Holding_B1']=test['Product_Holding_B1'].apply(eval)
test1=test['Product_Holding_B1'].apply(pd.Series)
test1.drop([4,5,6,7],axis=1,inplace=True)
test1=test1.fillna('0')
result1 = pd.concat([test, test1], axis=1)
result1

In [ ]:
#result1['len_Product_Holding_B1']=result1['Product_Holding_B1'].apply(lambda x: len(x))
#result['len_Product_Holding_B1']=result['Product_Holding_B1'].apply(lambda x: len(x))
#result1.head()

In [28]:
result1.head()

In [29]:
result1.info()

In [30]:
result1.isnull().sum()

In [31]:
sns.countplot(result1.City_Category)

In [32]:
sns.countplot(result1.Gender)

In [33]:
sns.countplot(result1.Is_Active)

In [34]:
sns.countplot(result1.Customer_Category)

In [35]:
sns.histplot(result1.Age)

In [36]:
sns.distplot(result1.Vintage)

In [37]:
def eda(df):
    df['Gender']=df['Gender'].replace({'Female':0,'Male':1})
    df['City_Category']=df['City_Category'].replace({'C1':0,'C2':1})
    df['Customer_Category']=df['Customer_Category'].replace({'S1':0,'S2':1,'S3':2})   
    return df

In [38]:
eda(result)
result.head()

In [39]:
eda(result1)
result1.head()

In [40]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report,average_precision_score,precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.datasets import load_digits
from sklearn.metrics import plot_confusion_matrix 
from sklearn.naive_bayes import MultinomialNB,ComplementNB

#from sklearn.metrics import classification_report

In [41]:
y=LabelEncoder()

In [42]:
result[3]=y.fit_transform(result[3])
result1[3]=y.fit_transform(result1[3])

In [43]:
result[0]=y.fit_transform(result[0])
result[1]=y.fit_transform(result[1])
result[2]=y.fit_transform(result[2])

result

In [44]:
result1[0]=y.fit_transform(result1[0])
result1[1]=y.fit_transform(result1[1])
result1[2]=y.fit_transform(result1[2])
result1

In [ ]:
#result['Product_Holding_B1']=y.fit_transform(result['Product_Holding_B1'])
#result.head()

In [ ]:
#result1['Product_Holding_B1']=y.fit_transform(result1['Product_Holding_B1'])
#result1.head()

In [45]:
features1=[col for col in result1.columns if col not in ['Customer_ID','Product_Holding_B1']]
features1

In [46]:
features = []
categorical_features = []
numerical_features = []

for feature in test.columns:
    features.append(feature)
    if test.dtypes[feature] == object or test.dtypes[feature] == 'int8':
        categorical_features.append(feature)
    else:
        numerical_features.append(feature)

plt.bar([1,2],[len(categorical_features),len(numerical_features)])
plt.xticks([1,2],('Categorical','Numerical'))
plt.show()

print("Categorical features:", len(categorical_features))
print("Numerical features:", len(numerical_features))


In [47]:
X=result[features1]
test_X=result1[features1]
result['Product_Holding_B2']=y.fit_transform(result['Product_Holding_B2'])
target=result['Product_Holding_B2']

In [48]:
#train_x,test_x,train_y,test_y=train_test_split(X,target,test_size=0.2,random_state=1,shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(X, target, random_state = 1, test_size = 0.3)

In [49]:
error_rate=[]
for i in range(1,100):
    knn = KNeighborsClassifier(n_neighbors=i)
    model = knn.fit(x_train,y_train)
    pred_i = knn.predict(x_test)
    error_rate.append(np.mean(pred_i != y_test))
plt.figure(figsize=(13,8))
plt.plot(range(1,100), error_rate, linestyle = 'dotted', marker = 'o',color = 'g')
plt.xlabel('K value')
plt.ylabel('Error Rate')
plt.title('K value Vs Error Rate')
plt.show()

In [50]:
model = KNeighborsClassifier(n_neighbors=30).fit(X,target)
#pred = model.predict(x_test)
#plot_confusion_matrix(model,x_test,y_test,cmap=plt.cm.Blues)
#print(classification_report(y_test,pred))  


In [52]:
preds = model.predict(test_X)
preds

In [53]:
sample_submission['Product_Holding_B2']=preds
sample_submission.head()

In [ ]:
sample_submission.to_csv('ghfj19.csv',index=False)

In [54]:
sample_submission['Product_Holding_B2']=y.inverse_transform(sample_submission['Product_Holding_B2'])
sample_submission.head()